## Inter IIT DevRev Team 11

#### Installations

#### Imports

In [ ]:
import os
import sys
import json
import yaml
import wandb
import pickle
import argparse
import pandas as pd

import torch
from torch.utils.data import DataLoader

from transformers import AutoTokenizer

from config import Config
from src import BaselineQA, FewShotQA_Model,BaselineClf
from utils import Trainer, set_seed, Retriever
from data import SQuAD_Dataset, SQuAD_Dataset_fewshot
from utils import build_tf_idf_wrapper, store_contents

from onnxruntime.transformers import optimizer as onnx_optimizer
import onnxruntime
import torch.onnx